In [1]:
# %pip install openai
# %pip install openpyxl
# %pip install python-dotenv

from IPython.display import clear_output
clear_output()

In [20]:
import json
import random
import re
import pandas as pd
import numpy as np
import pprint


# from sklearn.datasets import fetch_20newsgroups

# import nltk
# from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
Sample1 = """
For the PR I believe it's all good, the way to test it is on development mode in looker and make it work basically. The file is clean and all good for deployment.

But in terms of norming and future cases, I'm strongly against keeping this workflow if the dashboard has the possibility of receiving updates. 
I believe the use of LookML Dashboard was used almost correctly in this case, 
but on the case where we need to apply changes in this dashboard or add dimensions/new tiles it's not maintainable to add these things via LookML Dashboards overall, 
based on Looker is also a tool built to actually use the User-Defined features on explores. 
I believe we should follow more on the line where Guilherme explained to us, 
use this process to deploy the dashboard once it's ready and explore the options about changing sources when it's in production (model/views names...). 
On the other hand, Indeed this open us for huge improvements and version control on our dashboards, 
I strongly recommend exploring the possibility to convert this code into a User-Defined dashboard and see how we could keep changing it when its necessary (e.g. a new requirement comes).

(e.g. my opinion: We could talk more about this with the whole team and maybe leave this option open to whoever builds a dashboard if they want to have it as a LookML Dashboard if the dashboard requires it, 
like in this case, but evaluate why do we have to leave it like this; 
in this triage case we only did that and it cause us days of work just to have a dynamic hard coded date in the date selection because the business had hard times adapting with the Looker built-in options, 
we could have explored an option similar to what we do with the timeframes for example, and we never ended up discovering and exploring more LookML dashboards. 
It also worries me that this is a huge cannon to tackle minor requirements.)
Based on this, I think a discussion about it would be the way to go, it's a feature that Looker has, as any other, 
but it's not that straight forward to use, it limits us in other ways but it brings us a lot of other good stuff and robust and powerful opportunities, 
one of it is to keep version control about changes in the dashboard over time, but we have to decide as a team either is it is required/nice to have/cool thing let's use only when we need or desire.
"""

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_PROJECT = os.getenv("OPENAI_PROJECT_ID")
OPENAI_APIKEY = os.getenv("OPENAI_APIKEY")

# print(OPENAI_PROJECT,OPENAI_APIKEY)

## Authenticate with OpenAI via CURL
# !curl https://api.openai.com/v1/models -H "Authorization: Bearer $OPENAI_APIKEY" -H "OpenAI-Project: $OPENAI_PROJECT"

In [5]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_APIKEY,
    project=OPENAI_PROJECT
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful technical assistant, \
                                       specialist in non violent communication and all communication matters, \
                                       with the most updated infomation about IT and data science."}, 
        {"role": "user", "content":   "I'm an a NLP student and I need help with a feedback. \
                                       Tell me about what could be better on the following feedback to be more actionable, \
                                       and also in a scale of 1-10, how do rank this feedback (and explain based on what). Feedback:"+Sample1},
    ]
)

print(completion.choices[0].message)

#write a file with the response:

with open("resposta1.md", "w") as f:
    f.write(completion.choices[0].message.content)

ChatCompletionMessage(content="The feedback you've provided is detailed and brings up several important considerations, but there is room for improvement to make it more actionable. To enhance this feedback, consider the following suggestions:\n\n1. **Clarify the Objective:** Begin by clearly stating the purpose of the feedback. For instance, are you trying to improve a workflow, ensure future scalability, or both?\n\n2. **Be Specific:** Provide specific examples or scenarios where the current approach could fail. This will help the recipient understand the context and importance.\n\n3. **Actionable Steps:** Break down your suggestions into clear, actionable steps. Instead of just recommending broader exploration or discussion, outline what specific actions or experiments should be conducted.\n\n4. **Focus on Positives and Negatives:** Balance the discussion by highlighting what has been done well and what specifically needs improvement, ensuring a constructive tone.\n\n5. **Encourage 

In [13]:
# Dataset with studentes 
Sample2 = pd.read_excel('..\Datasets\TeachSatisfaction.xlsx')

In [14]:
Sample3_empathy = pd.read_csv('..\Datasets\empathy.csv')
Sample3_empathy.head()

,user_post,caregiver_post,empathetic_segments,empathy_class
0,I called in sick to work today. For the first ...,Is that really so bad? Maybe it was the smart ...,Hope you feel better soon|,1
1,i wanna jump from an elevated place. i wanna k...,I don't want you to jump. I don't want you to ...,I don't want you to jump. I don't want you to ...,1
2,All the people who will be kissed on New year'...,"Hey, just wait it out. I know what you feel. S...",if you ever wanna talk about anything I'd be m...,1
3,"The world is really pretty.. Pity, that I won'...","You're a good man, Median. I hope you know thi...","You're a good man|I'd rather you live, persona...",1
4,Don't know what this is.... Not sure if this i...,"Thats totally normal, most people get that fro...","Thats totally normal, most people get that fro...",1


In [33]:
Sample3_empathy.iloc[0]

pprint.pprint(['### User Post: ', 
               Sample3_empathy.iloc[0][0],
               '### Caregiver Post: ',
               Sample3_empathy.iloc[0][1],
               '### Empathetic Segments: ',
               Sample3_empathy.iloc[0][2],
               '### Empathy Class: ',
               Sample3_empathy.iloc[0][3]])

['### User Post: ',
 'I called in sick to work today. For the first time in a couple months I was '
 'feeling so Shitty I called into both my jobs today. Just feeling too burnt '
 'out, but I slept a lot, read quite a bit and finally managed to shower so '
 'the day a complete waste',
 '### Caregiver Post: ',
 'Is that really so bad? Maybe it was the smart decision because you needed '
 "that time to read recover. You're being kind to yourself when you need it "
 "and that's important. Hope you feel better soon.",
 '### Empathetic Segments: ',
 'Hope you feel better soon|',
 '### Empathy Class: ',
 1]


In [51]:
def print_empathy_sample(sample: pd.DataFrame, index: int):
    return pprint.pprint(['-','### User Post: ', 
                        sample.iloc[index][0],
                        '-','### Caregiver Post: ',
                        sample.iloc[index][1],
                        '-','### Empathetic Segments: ',
                        sample.iloc[index][2],
                        '-','### Empathy Class: ',
                        sample.iloc[index][3]])
    

print_empathy_sample(Sample3_empathy,2)

['-',
 '### User Post: ',
 "All the people who will be kissed on New year's eve. I'll be alone like "
 "usual I'll never get someone to kiss me It's fine No one understands how "
 'hopelessly, alone, and angrily some people live. It would scare some '
 "attractive people that haven't ever lived like that.",
 '-',
 '### Caregiver Post: ',
 "Hey, just wait it out. I know what you feel. Sometimes I think I'm "
 "attractive, and sometimes I can't even look at myself I'm so disgusted. I've "
 "come to realize it's all just a fucked up mind game I play with myself. I "
 "doubt you're actually that ugly, you're probably kinda cute honestly, but if "
 "you're anything like me, you've tricked yourself into thinking you're ugly. "
 'If you want people to want to be around you, you have to do some basic '
 'things like showering, putting on actual clothes, smiling, excercising a '
 "little and pretending you have confidence. It's honestly dumb, but these "
 'things can be hard to do, especially i